# UCI数据集的分类

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
# import seaborn as sns
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from __future__ import absolute_import, division
from urllib.request import urlopen
from zipfile import ZipFile
from pandas import DataFrame
from io import BytesIO
import pandas as pd
import numpy as np
import sys
import os

import logging

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from SFFS import SFFS
from numpy import linalg as LA
from xgboost import XGBClassifier

# We set the logger to Error level
# This is not recommend for normal use as you can oversee important Warning messages
logging.basicConfig(level=logging.ERROR)
_logger = logging.getLogger(__name__)

module_path = '/home/hadoop'
data_file_name = os.path.join(module_path, 'data')
data_file_bodyacc_x_dataset = os.path.join(module_path, 'data', 'UCI HAR Dataset', 'train', 'Inertial Signals','body_acc_x_train.txt')
data_file_bodyacc_y_dataset = os.path.join(module_path, 'data', 'UCI HAR Dataset', 'train', 'Inertial Signals','body_acc_y_train.txt')
data_file_bodyacc_z_dataset = os.path.join(module_path, 'data', 'UCI HAR Dataset', 'train', 'Inertial Signals','body_acc_z_train.txt')
data_file_bodygyro_x_dataset = os.path.join(module_path, 'data', 'UCI HAR Dataset', 'train', 'Inertial Signals','body_gyro_x_train.txt')
data_file_bodygyro_y_dataset = os.path.join(module_path, 'data', 'UCI HAR Dataset', 'train', 'Inertial Signals','body_gyro_y_train.txt')
data_file_bodygyro_z_dataset = os.path.join(module_path, 'data', 'UCI HAR Dataset', 'train', 'Inertial Signals','body_gyro_z_train.txt')
data_file_totalacc_x_dataset = os.path.join(module_path, 'data', 'UCI HAR Dataset', 'train', 'Inertial Signals','total_acc_x_train.txt')
data_file_totalacc_y_dataset = os.path.join(module_path, 'data', 'UCI HAR Dataset', 'train', 'Inertial Signals','total_acc_y_train.txt')
data_file_totalacc_z_dataset = os.path.join(module_path, 'data', 'UCI HAR Dataset', 'train', 'Inertial Signals','total_acc_z_train.txt')

data_file_name_classes = os.path.join(module_path, 'data', 'UCI HAR Dataset', 'train', 'y_train.txt')

data_path=[data_file_bodyacc_x_dataset, data_file_bodyacc_y_dataset, data_file_bodyacc_z_dataset,
          data_file_bodygyro_x_dataset, data_file_bodygyro_y_dataset, data_file_bodygyro_z_dataset,
          data_file_totalacc_x_dataset, data_file_totalacc_y_dataset, data_file_totalacc_z_dataset]

sensor_path={'body_acc_x':data_file_bodyacc_x_dataset, 
             'body_acc_y':data_file_bodyacc_y_dataset, 
             'body_acc_z':data_file_bodyacc_z_dataset, 
             'body_gyro_x':data_file_bodygyro_x_dataset, 
             'body_gyro_y':data_file_bodygyro_y_dataset, 
             'body_gyro_z':data_file_bodygyro_z_dataset, 
             'total_acc_x':data_file_totalacc_x_dataset, 
             'total_acc_y':data_file_totalacc_y_dataset, 
             'total_acc_z':data_file_totalacc_z_dataset}

def download_har_dataset():
    zipurl = 'https://github.com/MaxBenChrist/human-activity-dataset/blob/master/UCI%20HAR%20Dataset.zip?raw=true'

    if os.path.exists(data_file_bodyacc_x_dataset) and os.path.exists(data_file_name_classes):
        _logger.warning("You have already downloaded the Human Activity Data Set.")
        return

    with urlopen(zipurl) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(path=data_file_name)
        zfile.close()

def load_har_dataset(sensor_list, sampling):
    sensor_data={}
    try:
        for i in range(len(sensor_list)):
            tmp_data=np.loadtxt(sensor_path[sensor_list[i]])
            tmp_sample=tmp_data[:,0:64]            
            sensor_data[sensor_list[i]]=tmp_sample[:,::sampling]
            
        return sensor_data
    except IOError:
        raise IOError('File {} was not found. Have you downloaded the dataset with download_har_dataset() '
                      'before?'.format(data_file_name_dataset))
        
def load_har_feature_name():
    try:
        return pd.read_csv(data_file_name_features, delim_whitespace=True, header=None)
    except IOError:
        raise IOError('File {} was not found. Have you downloaded the dataset with download_har_dataset() '
                      'before?'.format(data_file_name_dataset))

def load_har_classes():
    try:
        return pd.read_csv(data_file_name_classes, delim_whitespace=True, header=None, squeeze=True)
    except IOError:
        raise IOError('File {} was not found. Have you downloaded the dataset with download_har_dataset() '
                      'before?'.format(data_file_name_classes))
        
def getHar(num=1000):
    df = load_har_dataset()[:num]
    featurename=load_har_feature_name()[:num]
    featurename=featurename.ix[:,1]
    return df, featurename

#根据Win，生成DataFrame格式中的Time列
def getTimeColumn(win):
    time=np.arange(win)
    for idx in range(1, sensornum):
        timetmp=np.arange(win)
        time=np.concatenate((time, timetmp), axis=0)
    time.shape=(len(time),1)
    return time

#根据Win和数据，生成DataFrame格式中的kind和value列
def getKindValueColumn(sd, win):
    for i in range(sensornum):
        kind=i*np.ones((win, 1),dtype=int)
        sensorcols=sd[i]
        sensorcols.shape=(win, 1)
        sdata=np.column_stack((kind, sensorcols))
        if i==0:
            sensorframe=sdata
        else:
            sensorframe=np.row_stack((sensorframe, sdata))
    return sensorframe

def getIdColumn(win, count):
    return count*np.ones((win*sensornum, 1))

def showIndex(intro="", i="", label="", fixed_row="", fixed_num="", start_row="", start_idx="", end_row="", end_idx=""):
    print("\n"+intro)
    print("i: "+ str(i))
    print("label: "+ str(label))
    print("fixed_row:"+str(fixed_row))
    print("fixed_num:"+str(fixed_num))
    print("start_row:"+str(start_row))
    print("start_idx:"+str(start_idx))
    print("end_row:"+str(end_row))
    print("end_idx:"+str(end_idx))

def getHarData(har_data, sampling, win, step, use_num):
    y = load_har_classes(); 
    sensor_data=har_data
    max_row=len(sensor_data[sensor[0]]); max_column=sensor_data[sensor[0]].shape[1]
    print(str(max_row))
    print(str(max_column))
              
    if step<=win:
        fixed_row=0; fixed_num=0; label=[];
        for i in range(use_num):
            sd=[]
            start_row=fixed_row + int(step*(i-fixed_num)/max_column)
            end_row=fixed_row + int((step*(i-fixed_num)+win)/max_column) if (step*(i-fixed_num)+win)%max_column!=0 \
                                                                else fixed_row + int((step*(i-fixed_num)+win)/max_column)-1    #这一行是能取到的
            if end_row<max_row:
                if y[start_row]==y[end_row]:
                    start_idx=(step*(i-fixed_num))%max_column
                    end_idx=max_column if (step*(i-fixed_num)+win)%max_column==0 else (step*(i-fixed_num)+win)%max_column    #这个end_idx是比能访问的idx多1的，取不到
#                     showIndex("if:", i, fixed_row, fixed_num, start_row, start_idx, end_row, end_idx)
                else:
                    while 1:
#                         showIndex("if:", i, label[i], fixed_row, fixed_num, start_row, start_idx, end_row, end_idx)
                        if y[start_row]==y[start_row+1]:
                            start_row+=1
                        else:
                            start_row+=1
                            fixed_row=start_row; fixed_num=i
                            end_row=fixed_row + int((step*(i-fixed_num)+win)/max_column) if (step*(i-fixed_num)+win)%max_column!=0 \
                                                                else fixed_row + int((step*(i-fixed_num)+win)/max_column)-1
                            start_idx=0
                            end_idx=max_column if (step*(i-fixed_num)+win)%max_column==0 else (step*(i-fixed_num)+win)%max_column    #这个end_idx是比能访问的idx多1的，取不到
#                             showIndex("while:", i, label[i], fixed_row, fixed_num, start_row, start_idx, end_row, end_idx)
                            break
            else:
                print('\033[1;33;48mNote:the real \'use_num\' is '+ str(i))
                break
            
            for j in range(sensornum):
                for row in range(start_row, end_row + 1):
                    for column in range(max_column):
                        if row==start_row:
                            if start_row!=end_row:
                                if column>=start_idx:
                                    sd.append(sensor_data[sensor[j]][row, column])
                            elif column>=start_idx and column<end_idx:
                                sd.append(sensor_data[sensor[j]][row, column])
                        elif row!=start_row and row!=end_row:
                            sd.append(sensor_data[sensor[j]][row, column])
                        elif row==end_row and column<end_idx:
                            sd.append(sensor_data[sensor[j]][row, column])                    
            
            label.append(y[start_row])  
#             showIndex("for down:", i, label[i], fixed_row, fixed_num, start_row, start_idx, end_row, end_idx)

            sd=np.array(sd); sd.shape=(len(sd), 1); sd=sd.reshape(-1, win)

            time=getTimeColumn(win)
            kindvalue=getKindValueColumn(sd, win)
            idary=getIdColumn(win, i)
            actionary=np.column_stack((idary,time, kindvalue ))
            
            if i == 0:
                dataarray=actionary
            else:
                dataarray=np.concatenate((dataarray, actionary), axis=0)
        label=pd.Series(label)
        return dataarray, label
    else:
        raise IOError('\'step\' of slide window shoud be less than \'win\'')
        
################ 人工提取特征 ##################
def ftrExt(data, fs, step):
    data['MI'] = np.array([])
    data['SMA'] = np.array([])
    #data['Velo'] = np.array([])
    for i in xrange(0, data['body_acc_x'].shape[0]):
        tmp = np.concatenate((np.reshape(data['body_acc_x'][i], (-1,1)), 
                              np.reshape(data['body_acc_y'][i], (-1,1)), 
                              np.reshape(data['body_acc_z'][i], (-1,1))), axis=1)
        mi = LA.norm(tmp, 2, axis=1)
        sma = np.sum(np.abs(tmp), 1)
        velo = calcVelo(tmp[:,:2], fs, step)
        data['MI'] = np.append(data['MI'],mi)
        data['SMA'] = np.append(data['SMA'],sma)
        #data['Velo'] = np.append(data['Velo'],velo)
    data['MI'] = np.reshape(data['MI'], (-1, step))
    data['SMA'] = np.reshape(data['SMA'], (-1, step))
    #data['Velo'] = np.reshape(data['Velo'], (-1, step))
    sensor = ["body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", "MI", "SMA"]
    return data, sensor
    
################ Calculate Velocity ###################

def calcVelo(acc, fs, win):
    velo = np.zeros(acc.shape)
    for i in xrange(0, acc.shape[0]):
        if i % win: 
            velo[i] = velo[i-1] + (acc[i-1]+acc[i])/(2*fs)
    return LA.norm(velo, 2, axis=1)

In [2]:
# 六个传感器数据
# sensor=["body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z"]
sensor=["body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z"]

# 一些常量，这里不需要filenum，文件数量是根据sensor的数量走的
sensornum=len(sensor)
#窗口步长大小
sampling=1
win=128
step=64      #步长应该小于等于win
fs = 50/sampling

har_data=load_har_dataset(sensor, sampling)

########### Feature Extract ##############
#har_data, sensor = ftrExt(har_data, fs, step)
#sensornum=len(sensor)
##########################################

max_row=len(har_data[sensor[0]]); max_column=har_data[sensor[0]].shape[1]
                                                            
print('\033[1;33;48mNote: you can try the max \'use_num\': '+ str(int((max_column*max_row-win)/step+1)) 
      + "\n"+"But the real \'use_num\' may be less than it ")
# df, featurename=getHar()
#使用的样本数量(一个窗口的数据是一个样本)，建议设置陈6的倍数
use_num=7350

data, label=getHarData(har_data, sampling, win, step, use_num)

master_df= DataFrame(data, columns=['id', 'time', 'kind', 'value'])

#print(data)
# print(master_df)
# print(label.shape)

Note: you can try the max 'use_num': 7351
But the real 'use_num' may be less than it 
7352
64
Note:the real 'use_num' is 7072


In [3]:
# extraction_settings = ComprehensiveFCParameters()
#extraction_settings = EfficientFCParameters()
# extraction_settings = MinimalFCParameters()
extraction_settings = {'ar_coefficient': [{'coeff': 0, 'k': 10},
    {'coeff': 1, 'k': 10},
    {'coeff': 2, 'k': 10},
    {'coeff': 3, 'k': 10},
    {'coeff': 4, 'k': 10}],
          'autocorrelation': [{'lag': 0},{'lag': 1},{'lag': 2},{'lag': 3},{'lag': 4},
                              {'lag': 5},{'lag': 6},{'lag': 7},{'lag': 8},{'lag': 9}],
          'quantile': [{'q': 0.1},{'q': 0.2},{'q': 0.3},{'q': 0.4},
                       {'q': 0.6},{'q': 0.7},{'q': 0.8},{'q': 0.9}],
          'number_peaks': [{'n': 1}, {'n': 3}, {'n': 5}],
          'minimum': None,
          'maximum': None,
          'median': None,
          'sum_values': None}

%time X = extract_features(master_df, default_fc_parameters=extraction_settings, column_id='id', column_sort="time", column_kind="kind", column_value="value")
impute(X)

Feature Extraction: 100%|██████████| 6/6 [03:22<00:00, 32.30s/it]


CPU times: user 1.11 s, sys: 235 ms, total: 1.35 s
Wall time: 3min 23s


,0.0__quantile__q_0.9,0.0__quantile__q_0.8,0.0__quantile__q_0.1,0.0__quantile__q_0.3,0.0__quantile__q_0.2,0.0__number_peaks__n_5,0.0__quantile__q_0.4,0.0__quantile__q_0.7,0.0__quantile__q_0.6,0.0__maximum,...,5.0__quantile__q_0.3,5.0__quantile__q_0.1,5.0__quantile__q_0.6,5.0__quantile__q_0.7,5.0__quantile__q_0.4,5.0__ar_coefficient__k_10__coeff_0,5.0__ar_coefficient__k_10__coeff_1,5.0__ar_coefficient__k_10__coeff_2,5.0__ar_coefficient__k_10__coeff_3,5.0__ar_coefficient__k_10__coeff_4
id,,,,,,,,,,,,,,,,,,,,,
0.0,0.006216,0.005042,-0.000954,0.000009,-0.000411,7.0,0.000528,0.004334,0.002974,0.010810,...,0.006358,0.002399,0.011166,0.013151,0.007866,0.000975,1.585430,-1.278527,1.076990,-1.195126
1.0,0.002598,0.001562,-0.001828,-0.000494,-0.001094,10.0,-0.000243,0.001041,0.000574,0.005251,...,0.002050,-0.004545,0.008383,0.010178,0.004780,0.000447,2.128164,-2.519096,2.685167,-2.844782
2.0,0.003382,0.002552,-0.003149,-0.000417,-0.001501,11.0,0.000082,0.001829,0.001079,0.008167,...,-0.002257,-0.005545,0.005872,0.009645,-0.000173,0.000527,2.019692,-2.293839,2.442846,-2.495636
3.0,0.002985,0.002261,-0.002535,-0.000895,-0.001597,9.0,-0.000150,0.001480,0.000833,0.008167,...,-0.004453,-0.009005,-0.000955,0.000383,-0.002731,-0.000297,1.797571,-2.071388,2.387350,-2.493283
4.0,0.002197,0.001513,-0.002608,-0.001255,-0.001737,8.0,-0.000872,0.000879,0.000371,0.005650,...,-0.008615,-0.013390,-0.001874,-0.000975,-0.006212,-0.000434,2.023739,-2.277023,2.766367,-3.117022
5.0,0.002422,0.001830,-0.003185,-0.001143,-0.001766,8.0,-0.000638,0.001144,0.000547,0.005650,...,-0.010039,-0.017491,0.000969,0.003091,-0.006699,-0.000051,2.042735,-2.167073,2.418767,-2.573684
6.0,0.003801,0.002180,-0.002577,-0.000948,-0.001747,8.0,-0.000354,0.001843,0.001027,0.006637,...,-0.002087,-0.011789,0.006236,0.010362,0.001715,0.000393,2.147980,-2.612421,3.063679,-3.343618
7.0,0.003352,0.002216,-0.003145,-0.001659,-0.002167,8.0,-0.001116,0.001532,0.000624,0.006637,...,-0.007071,-0.011515,0.008788,0.013248,-0.002930,0.000014,2.050688,-2.305927,2.664509,-2.848220
8.0,0.002728,0.001877,-0.002903,-0.001357,-0.001789,9.0,-0.000782,0.001055,0.000441,0.006897,...,-0.009181,-0.012382,-0.003390,-0.001394,-0.007598,-0.000743,1.941570,-2.164241,2.327152,-2.529690


In [4]:
for n in X.columns:
    std = np.std(X[n])
    X[n] = (X[n]-np.mean(X[n]))/std if std>0 else X[n]-np.mean(X[n])

In [5]:
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
X_train_fs, X_val, y_train_fs, y_val = train_test_split(X_train, y_train, test_size=.125)

## XGBoost分类器训练

In [6]:
eval_set = [(X_val, y_val)]
cl = XGBClassifier(max_depth=5, n_estimators=200, objective='multi:softmax', reg_lambda=2)
cl.fit(X_train, y_train, eval_metric='mlogloss', eval_set=eval_set, early_stopping_rounds=30, verbose=False)
score = accuracy_score(y_test, cl.predict(X_test))
print(score)

0.958303886926


In [7]:
mean = 0
for i in xrange(0, 20):
    X_train_tmp, X_test, y_train_tmp, y_test = train_test_split(X, y, test_size=.2)
    X_train, X_val, y_train, y_val = train_test_split(X_train_tmp, y_train_tmp, test_size=.125)
    eval_set = [(X_val, y_val)]
    cl = XGBClassifier(max_depth=5, n_estimators=200, objective='multi:softmax', reg_lambda=2)
    cl.fit(X_train, y_train, eval_metric='mlogloss', eval_set=eval_set, early_stopping_rounds=30, verbose=False)
    score = accuracy_score(y_test, cl.predict(X_test))
    print('Loop %d, accuracy=%f' % (i, score))
    mean += score
print('Mean accuracy is %f' % (mean/20))
    

Loop 0, accuracy=0.945583
Loop 1, accuracy=0.951943
Loop 2, accuracy=0.947703
Loop 3, accuracy=0.956890
Loop 4, accuracy=0.944170
Loop 5, accuracy=0.951943
Loop 6, accuracy=0.948410
Loop 7, accuracy=0.953357
Loop 8, accuracy=0.941343
Loop 9, accuracy=0.955477
Loop 10, accuracy=0.951237
Loop 11, accuracy=0.954770
Loop 12, accuracy=0.950530
Loop 13, accuracy=0.952650
Loop 14, accuracy=0.956890
Loop 15, accuracy=0.956890
Loop 16, accuracy=0.941343
Loop 17, accuracy=0.947703
Loop 18, accuracy=0.942049
Loop 19, accuracy=0.952650
Mean accuracy is 0.950177
